In [2]:
import sys
import os
import pandas as pd
import numpy as np
sys.path.append('/nfs/research1/zerbino/jhidalgo/inteql/inteql/')
from modelFunctions import *
from scipy.stats import ttest_ind
from scipy import stats
import random

In [ ]:
##### LOAD eQTLs #######
"""
Pre-finemapped eQTLs from GTEX database are loaded to make predictions based on attributed weights for each tissue.
Only CaVEMaN dataset is being used here although a consensus dataset is recommended.
"""

In [371]:
eqtl_file = '/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/pre-finemapped/GTEx_v8_finemapping_CaVEMaN/GTEx_v8_finemapping_CaVEMaN.txt.gz'

In [372]:
eQTLs=pd.read_csv(eqtl_file,sep='\t',compression='gzip')

FileNotFoundError: [Errno 2] No such file or directory: '/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/data/original-data/pre-finemapped/GTEx_v8_finemapping_CaVEMaN/GTEx_v8_finemapping_CaVEMaN.txt.gz'

In [5]:
eQTLs=eQTLs.pivot_table(index=['eQTL','GENE'],columns='TISSUE',values='Probability').reset_index()

In [6]:
eQTLs

,TISSUE,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,Brain_Caudate_basal_ganglia,Brain_Cerebellar_Hemisphere,...,Skin_Not_Sun_Exposed_Suprapubic,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood
eQTL,GENE,,,,,,,,,,,,,,,,,,,,,
chr10_100007241_C_T,ENSG00000107554.16,NaN,NaN,NaN,NaN,NaN,0.201558,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chr10_100008640_A_G,ENSG00000107554.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chr10_100009635_T_G,ENSG00000107554.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chr10_100014923_C_T,ENSG00000119929.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chr10_100017063_G_A,ENSG00000107554.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX_99899260_C_G,ENSG00000233680.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.231066,NaN,NaN,NaN,NaN,NaN
chrX_99953318_T_G,ENSG00000102362.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chrX_99959994_A_G,ENSG00000101811.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.261396,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
### Raw eQTL prediction with RF, mean and dummy ###

In [101]:
def random_forest_regressor_local(data, X_label, y_label, random_state, test_size=0.3, max_depth=83, n_estimators=136,
                            min_samples_split=2, min_samples_leaf=1, max_features='sqrt'):
    # Split data
    X = data[X_label]
    y = data[y_label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Create a model and train it
    model = RandomForestRegressor(random_state=random_state, max_depth=max_depth, n_estimators=n_estimators)
    model = model.fit(X_train, y_train)

    # Predict values for test set and asses error
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    slope, intercept, r_value, p_value, std_err = linregress(y_test, y_pred)
    importances = list(model.feature_importances_)
    feature_list = list(X.columns)
    feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
    feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)
    # Return result object
    return {'rmse': rmse, 'r_value': r_value, 'feature_importances': feature_importances}

In [208]:
def dummy_regressor_local(data, X_label, y_label, random_state, test_size=0.3):
    # Split data
    X = data[X_label]
    y = data[y_label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Create a model and train it
    model = DummyRegressor()
    model = model.fit(X_train, y_train)

    # Predict values for test set and asses error
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    slope, intercept, r_value, p_value, std_err = linregress(y_test, y_pred)


    # Return result object
    return {'rmse': rmse,'r_value': r_value}

In [103]:
def mean_regressor_local(data,X_label,y_label):
    X = data[X_label]
    y = data[y_label]
    y_pred = X.mean(axis=1)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    slope, intercept, r_value, p_value, std_err = linregress(y, y_pred)
    return {'rmse': rmse, 'r_value': r_value}

In [104]:
random_state = 42

In [209]:
random_forest={}
dummy={}
mean={}
# i=eQTLs.columns[21]
for i in eQTLs.columns:
    print('Starting with:',i,end='\r')
    features=eQTLs.columns.drop(i)
    data=eQTLs[~ eQTLs.drop(columns=i).isna().all(axis=1) & eQTLs[i].notna()]
    random_forest[i] = random_forest_regressor_local(data.fillna(0), features, i, random_state=random_state)
    dummy[i] = dummy_regressor_local(data.fillna(0), features, i, random_state)
    mean[i] = mean_regressor_local(data, features, i)
    print("{} {} {:.2f} {} {:.2f} {} {:.2f} {} {}".format(i,'processed. RF RMSE:',random_forest[i]['rmse'],'Mean RMSE:',mean[i]['rmse'],'Dummy RMSE:',dummy[i]['rmse'],'Data points:',len(data)),flush=True)
    #print(i,'processed. RF RMSE:',random_forest[i]['rmse'],'Mean RMSE:',mean[i]['rmse'],'Dummy RMSE:',dummy[i]['rmse'],flush=True)

Adipose_Subcutaneous processed. RF RMSE: 0.20 Mean RMSE: 0.21 Dummy RMSE: 0.27 Data points: 11616
Adipose_Visceral_Omentum processed. RF RMSE: 0.19 Mean RMSE: 0.20 Dummy RMSE: 0.26 Data points: 9307
Adrenal_Gland processed. RF RMSE: 0.17 Mean RMSE: 0.19 Dummy RMSE: 0.23 Data points: 4832
Artery_Aorta processed. RF RMSE: 0.19 Mean RMSE: 0.19 Dummy RMSE: 0.26 Data points: 8788
Artery_Coronary processed. RF RMSE: 0.16 Mean RMSE: 0.20 Dummy RMSE: 0.23 Data points: 4159
Artery_Tibial processed. RF RMSE: 0.21 Mean RMSE: 0.22 Dummy RMSE: 0.27 Data points: 11265
Brain_Amygdala processed. RF RMSE: 0.14 Mean RMSE: 0.18 Dummy RMSE: 0.21 Data points: 2183
Brain_Anterior_cingulate_cortex_BA24 processed. RF RMSE: 0.15 Mean RMSE: 0.18 Dummy RMSE: 0.21 Data points: 3224
Brain_Caudate_basal_ganglia processed. RF RMSE: 0.15 Mean RMSE: 0.18 Dummy RMSE: 0.22 Data points: 4947
Brain_Cerebellar_Hemisphere processed. RF RMSE: 0.17 Mean RMSE: 0.18 Dummy RMSE: 0.24 Data points: 5560
Brain_Cerebellum processed.

In [242]:
pd.DataFrame(random_forest).loc[['rmse']].merge(pd.DataFrame(mean).loc[['rmse']],how='outer').merge(pd.DataFrame(dummy).loc[['rmse']],how='outer').rename(index={0:'RF RMSE',1:'Mean RMSE',2:'Dummy RMSE'}).round(decimals=3).T.merge(pd.DataFrame(random_forest).loc[['r_value']].merge(pd.DataFrame(mean).loc[['r_value']],how='outer').merge(pd.DataFrame(dummy).loc[['r_value']],how='outer').rename(index={0:'RF R',1:'Mean R',2:'Dummy R'}).round(decimals=3).T,left_index=True,right_index=True)

,RF RMSE,Mean RMSE,Dummy RMSE,RF R,Mean R,Dummy R
Adipose_Subcutaneous,0.199,0.210,0.274,0.690,0.675,-0.0
Adipose_Visceral_Omentum,0.191,0.196,0.263,0.688,0.678,0.0
Adrenal_Gland,0.173,0.190,0.230,0.661,0.668,-0.0
Artery_Aorta,0.190,0.193,0.259,0.681,0.677,0.0
Artery_Coronary,0.158,0.196,0.228,0.721,0.686,0.0
Artery_Tibial,0.205,0.219,0.274,0.666,0.644,0.0
Brain_Amygdala,0.137,0.181,0.206,0.746,0.718,0.0
Brain_Anterior_cingulate_cortex_BA24,0.146,0.177,0.213,0.729,0.704,0.0
Brain_Caudate_basal_ganglia,0.146,0.178,0.219,0.748,0.683,0.0
Brain_Cerebellar_Hemisphere,0.166,0.178,0.236,0.715,0.702,0.0


In [243]:
pd.DataFrame(random_forest).loc[['rmse']].merge(pd.DataFrame(mean).loc[['rmse']],how='outer').merge(pd.DataFrame(dummy).loc[['rmse']],how='outer').rename(index={0:'RF RMSE',1:'Mean RMSE',2:'Dummy RMSE'}).round(decimals=3).T.merge(pd.DataFrame(random_forest).loc[['r_value']].merge(pd.DataFrame(mean).loc[['r_value']],how='outer').merge(pd.DataFrame(dummy).loc[['r_value']],how='outer').rename(index={0:'RF R',1:'Mean R',2:'Dummy R'}).round(decimals=3).T,left_index=True,right_index=True).to_csv('/nfs/research1/zerbino/jhidalgo/inteql_GTEX_v8/plots/data/raw_eqtl_prediction.csv',index=True)

In [245]:
### Data processing ###

In [322]:
data=eQTLs.reset_index()

In [323]:
data['chr']=data['eQTL'].str.split("_",expand=True)[0].str.lstrip('chr')

In [324]:
data['pos']=data['eQTL'].str.split("_",expand=True)[1].str.lstrip('chr')

In [325]:
data['R']=data['eQTL'].str.split("_",expand=True)[2].str.lstrip('chr')

In [326]:
data['A']=data['eQTL'].str.split("_",expand=True)[3].str.lstrip('chr')

In [327]:
data=data.replace('X','23')

In [328]:
from sklearn import preprocessing

In [329]:
le = preprocessing.LabelEncoder()

In [330]:
le.fit(data['R'])

LabelEncoder()

In [331]:
data['R']=le.transform(data['R'])

In [332]:
le.fit(data['A'])

LabelEncoder()

In [333]:
data['A']=le.transform(data['A'])

In [369]:
i=eQTLs.columns[21]
#for i in eQTLs.columns:
print('Starting with:',i,end='\r')
features=data.columns[data.columns.to_list().index('Adipose_Subcutaneous'):].drop(i)
prepdata=data[~ data[data.columns[data.columns.to_list().index('Adipose_Subcutaneous'):prepdata.columns.to_list().index('Whole_Blood')]].drop(columns=i).isna().all(axis=1) & data[i].notna()]
random_forest_2 = random_forest_regressor_local(prepdata.fillna(0), features, i, random_state=random_state)
dummy_2 = dummy_regressor_local(prepdata.fillna(0), features, i, random_state)
mean_2 = mean_regressor_local(prepdata, data.columns[data.columns.to_list().index('Adipose_Subcutaneous'):prepdata.columns.to_list().index('Whole_Blood')].drop(i), i)
print("{} {} {:.3f} {} {:.3f} {} {:.3f} {} {}".format(i,'processed. RF RMSE:',random_forest_2['rmse'],'Mean RMSE:',mean_2['rmse'],'Dummy RMSE:',dummy_2['rmse'],'Data points:',len(prepdata)),flush=True)

Cells_EBV-transformed_lymphocytes processed. RF RMSE: 0.179 Mean RMSE: 0.199 Dummy RMSE: 0.214 Data points: 2313


In [362]:
features

Index(['Adipose_Subcutaneous', 'Adipose_Visceral_Omentum', 'Adrenal_Gland',
       'Artery_Aorta', 'Artery_Coronary', 'Artery_Tibial', 'Brain_Amygdala',
       'Brain_Anterior_cingulate_cortex_BA24', 'Brain_Caudate_basal_ganglia',
       'Brain_Cerebellar_Hemisphere', 'Brain_Cerebellum', 'Brain_Cortex',
       'Brain_Frontal_Cortex_BA9', 'Brain_Hippocampus', 'Brain_Hypothalamus',
       'Brain_Nucleus_accumbens_basal_ganglia', 'Brain_Putamen_basal_ganglia',
       'Brain_Spinal_cord_cervical_c-1', 'Brain_Substantia_nigra',
       'Breast_Mammary_Tissue', 'Cells_Cultured_fibroblasts', 'Colon_Sigmoid',
       'Colon_Transverse', 'Esophagus_Gastroesophageal_Junction',
       'Esophagus_Mucosa', 'Esophagus_Muscularis', 'Heart_Atrial_Appendage',
       'Heart_Left_Ventricle', 'Kidney_Cortex', 'Liver', 'Lung',
       'Minor_Salivary_Gland', 'Muscle_Skeletal', 'Nerve_Tibial', 'Ovary',
       'Pancreas', 'Pituitary', 'Prostate', 'Skin_Not_Sun_Exposed_Suprapubic',
       'Skin_Sun_Exposed_Lower

In [231]:
print('Starting with:',i,end='\r')
features=eQTLs.columns.drop(i)
data=eQTLs[~ eQTLs.drop(columns=i).isna().all(axis=1) & eQTLs[i].notna()]
print(random_forest_regressor_local(data.fillna(0), features, i, random_state=random_state))
print(dummy_regressor_local(data.fillna(0), features, i, random_state))
print(mean_regressor_local(data, features, i))

{'rmse': 0.19870644436534984, 'r_value': 0.6902867851534324, 'feature_importances': [('Nerve_Tibial', 0.23), ('Adipose_Visceral_Omentum', 0.14), ('Breast_Mammary_Tissue', 0.1), ('Skin_Sun_Exposed_Lower_leg', 0.07), ('Artery_Tibial', 0.05), ('Thyroid', 0.04), ('Lung', 0.03), ('Artery_Aorta', 0.02), ('Cells_Cultured_fibroblasts', 0.02), ('Colon_Transverse', 0.02), ('Esophagus_Gastroesophageal_Junction', 0.02), ('Esophagus_Mucosa', 0.02), ('Esophagus_Muscularis', 0.02), ('Heart_Atrial_Appendage', 0.02), ('Heart_Left_Ventricle', 0.02), ('Muscle_Skeletal', 0.02), ('Skin_Not_Sun_Exposed_Suprapubic', 0.02), ('Adrenal_Gland', 0.01), ('Artery_Coronary', 0.01), ('Brain_Cerebellum', 0.01), ('Colon_Sigmoid', 0.01), ('Pancreas', 0.01), ('Pituitary', 0.01), ('Prostate', 0.01), ('Spleen', 0.01), ('Stomach', 0.01), ('Testis', 0.01), ('Whole_Blood', 0.01), ('Brain_Amygdala', 0.0), ('Brain_Anterior_cingulate_cortex_BA24', 0.0), ('Brain_Caudate_basal_ganglia', 0.0), ('Brain_Cerebellar_Hemisphere', 0.0), 

In [2]:
###### Load RNA-seq data #######

In [2]:
os.chdir('../inteql/data/original-data/RNA-seq/')

In [3]:
original_data=pd.read_csv('Top_genes_ML.csv').rename(columns={'0':'Tissue'})

In [4]:
original_data

,Tissue,ENST00000538644.1,ENST00000575363.1,ENST00000475226.1,ENST00000426995.2,ENST00000398824.2,ENST00000256104.4,ENST00000237014.3,ENST00000594907.1,ENST00000472523.1,...,ENST00000251595.6,ENST00000370456.4,ENST00000493888.1,ENST00000376335.3,ENST00000221847.5,ENST00000376673.3,ENST00000524255.1,ENST00000494569.1,ENST00000354822.5,ENST00000446581.1
0,Adipose - Subcutaneous,0.00,0.00,5.67,0.38,0.04,1100.00,0.68,0.35,17.95,...,289.50,0.00,3.01,0.05,1.19,0.71,0.28,0.33,0.00,0.0
1,Muscle - Skeletal,0.30,0.00,1.27,389.50,0.02,115.10,0.62,0.00,9.65,...,154.60,0.16,0.89,0.00,0.22,0.00,0.00,0.03,0.00,0.0
2,Artery - Tibial,0.00,0.00,10.13,0.36,0.00,129.20,1.50,0.00,0.14,...,170.20,0.25,0.54,0.02,0.50,0.44,0.19,0.00,0.00,0.0
3,Artery - Coronary,0.19,0.78,63.44,0.74,0.00,3925.00,0.82,0.00,0.00,...,4689.00,0.00,0.29,0.02,1.04,0.00,0.00,0.00,0.03,0.0
4,Heart - Atrial Appendage,0.00,0.00,36.72,0.30,0.15,181.20,0.64,0.00,0.00,...,1356.00,0.05,0.05,0.03,0.95,0.00,0.57,0.00,0.00,469.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11683,Ovary,1.95,0.00,1.21,0.27,0.07,1.98,0.85,0.00,0.00,...,5.90,0.09,0.00,0.00,0.71,0.70,0.37,0.00,0.03,0.0
11684,Vagina,1.64,0.00,0.00,0.00,1.76,1.29,0.93,0.00,0.00,...,10.07,77.56,0.81,0.02,0.62,0.00,0.37,0.00,0.00,0.0
11685,Artery - Tibial,0.00,0.00,15.13,0.35,0.03,11.35,1.01,0.00,0.10,...,447.50,0.16,0.00,0.03,3.19,0.00,0.35,0.00,0.00,0.0
11686,Muscle - Skeletal,0.18,0.00,0.00,193.00,0.01,42.31,0.43,0.00,1.09,...,10.95,0.16,0.00,0.00,0.49,0.00,0.00,0.04,0.00,0.0


In [5]:
extended_data=pd.read_csv('Extended_data.csv',).rename(columns={'SMTSD':'Tissue'})

In [6]:
extended_data

,Tissue,ENST00000336375.5,ENST00000570029.1,ENST00000023064.4,ENST00000587772.1,ENST00000520188.1,ENST00000283752.5,ENST00000538787.1,ENST00000359354.2,ENST00000262502.5,...,ENST00000538107.1,ENST00000377871.3,ENST00000421062.2,ENST00000458333.2,ENST00000528160.1,ENST00000376891.4,ENST00000533330.2,ENST00000341074.5,ENST00000413673.1,ENST00000498496.1
0,Adipose - Subcutaneous,0.21,0.0,0.00,0.19,0.00,0.06,0.00,0.00,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00
1,Muscle - Skeletal,0.10,0.0,0.00,0.00,0.00,2.18,0.71,0.00,0.0,...,0.00,0.0,0.0,0.0,0.16,0.0,0.0,0.13,0.09,0.00
2,Artery - Tibial,0.09,0.0,0.00,0.00,0.00,1.14,0.00,0.00,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.33,0.00,0.00
3,Artery - Coronary,0.32,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00
4,Heart - Atrial Appendage,0.13,0.0,0.00,0.00,0.15,0.05,0.00,0.00,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11683,Ovary,0.77,0.0,0.00,0.40,0.00,0.23,0.15,0.00,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00
11684,Vagina,9.61,0.0,0.00,0.19,0.00,989.30,13.31,0.11,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,108.60,1.89,0.71
11685,Artery - Tibial,0.16,0.0,0.01,0.00,0.00,0.00,0.33,0.00,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.06
11686,Muscle - Skeletal,0.19,0.0,0.00,0.00,0.00,0.00,6.22,0.00,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00


In [7]:
data=original_data.merge(extended_data.drop(columns='Tissue'),left_index=True,right_index=True)

In [8]:
data=data.set_index('Tissue')

In [9]:
reindex_data=data.reset_index()

In [10]:
data

,ENST00000538644.1,ENST00000575363.1,ENST00000475226.1,ENST00000426995.2,ENST00000398824.2,ENST00000256104.4,ENST00000237014.3,ENST00000594907.1,ENST00000472523.1,ENST00000420959.2,...,ENST00000538107.1,ENST00000377871.3,ENST00000421062.2,ENST00000458333.2,ENST00000528160.1,ENST00000376891.4,ENST00000533330.2,ENST00000341074.5,ENST00000413673.1,ENST00000498496.1
Tissue,,,,,,,,,,,,,,,,,,,,,
Adipose - Subcutaneous,0.00,0.00,5.67,0.38,0.04,1100.00,0.68,0.35,17.95,0.00,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00
Muscle - Skeletal,0.30,0.00,1.27,389.50,0.02,115.10,0.62,0.00,9.65,0.00,...,0.00,0.0,0.0,0.0,0.16,0.0,0.0,0.13,0.09,0.00
Artery - Tibial,0.00,0.00,10.13,0.36,0.00,129.20,1.50,0.00,0.14,0.00,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.33,0.00,0.00
Artery - Coronary,0.19,0.78,63.44,0.74,0.00,3925.00,0.82,0.00,0.00,0.00,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00
Heart - Atrial Appendage,0.00,0.00,36.72,0.30,0.15,181.20,0.64,0.00,0.00,0.00,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ovary,1.95,0.00,1.21,0.27,0.07,1.98,0.85,0.00,0.00,0.24,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00
Vagina,1.64,0.00,0.00,0.00,1.76,1.29,0.93,0.00,0.00,0.00,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,108.60,1.89,0.71
Artery - Tibial,0.00,0.00,15.13,0.35,0.03,11.35,1.01,0.00,0.10,0.24,...,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.06


In [8]:
####### Prediction functions ######

In [38]:
def t_test (i):
    results_sample_p_val={}
    results_tissue_p_val={}
    results_count={}
    binary={}
    for Transc in data.columns:
        results_sample_p_val[Transc]={}
        results_tissue_p_val[Transc]={}
        results_count[Transc]={}
        binary[Transc]={}
        for Tissue in data.index.unique():
            ttest_sample=stats.ttest_1samp(data[Transc][Tissue],data.iloc[i][Transc])#######
            ttest_tissue=ttest_ind(data[Transc][Tissue],data[Transc][data.index != Tissue])
            results_sample_p_val[Transc][Tissue]=ttest_sample.pvalue
            results_tissue_p_val[Transc][Tissue]=ttest_tissue.pvalue
            binary[Transc][Tissue]=np.nan
            if ttest_sample.pvalue >= 0.0001 and ttest_tissue.pvalue <= 0.00001: binary[Transc][Tissue]= 1## 
            if ttest_sample.pvalue <= 0.0001 and ttest_tissue.pvalue <= 0.00001: binary[Transc][Tissue]= 0##
            if ttest_tissue.pvalue <= 0.00001: 
                results_count[Transc][Tissue]= 1
            else:
                results_count[Transc][Tissue]= 0
                
    return {'tissue':results_tissue_p_val,'transcript':results_sample_p_val,'binary':binary,'count':results_count}

In [10]:
def t_test_ratio (i):
    value={}
    for Transc in data.columns:
        value[Transc]={}
        for Tissue in data.index.unique():
            ttest_sample=stats.ttest_1samp(data[Transc][Tissue],data.iloc[i][Transc])
            value[Transc][Tissue]=np.nan
            if ttest_sample.pvalue >= 0.0001: value[Transc][Tissue] = ratiodata[Transc][Tissue]## 
    return {'value':value}

In [11]:
def t_test_ratio_external (i):
    value={}
    for Transc in data.columns:
        value[Transc]={}
        for Tissue in data.index.unique():
            ttest_sample=stats.ttest_1samp(data[Transc][Tissue],i[Transc])
            value[Transc][Tissue]=np.nan
            if ttest_sample.pvalue >= 0.0001: value[Transc][Tissue] = ratiodata[Transc][Tissue]## 
    return {'value':value}

In [12]:
##### LOAD eQTLs #######

In [13]:
eqtl_file = '/nfs/research1/zerbino/jhidalgo/inteql/data/output/output05.csv.gz'

In [14]:
eQTLs=pd.read_csv(eqtl_file,sep=',',compression='gzip')

In [15]:
eQTLs

,variant_id,gene_id,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta,Artery_Coronary,Artery_Tibial,Brain_Amygdala,Brain_Anterior_cingulate_cortex_BA24,...,Skin_Sun_Exposed_Lower_leg,Small_Intestine_Terminal_Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole_Blood,Cells_EBV-transformed_lymphocytes
0,19_24221728_G_C_b37,ENSG00000213096,0.263925,NaN,NaN,0.263636,NaN,0.203748,NaN,NaN,...,0.386554,NaN,NaN,0.249301,NaN,NaN,NaN,NaN,NaN,NaN
1,17_79670273_T_C_b37,ENSG00000182446,-0.105845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.231742,-0.194333,NaN,NaN,NaN,NaN
2,1_151056305_C_T_b37,ENSG00000197622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2_179580434_A_G_b37,ENSG00000223960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.717794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5_61605063_C_A_b37,ENSG00000068796,NaN,NaN,NaN,0.254959,NaN,0.154943,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407341,4_68898861_T_G_b37,ENSG00000198092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.142926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5407342,1_155415181_G_A_b37,ENSG00000203761,-0.228117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.226778,NaN,NaN,NaN,NaN,-0.337073,NaN,NaN,NaN,NaN
5407343,4_77413179_G_A_b37,ENSG00000163749,NaN,NaN,NaN,-0.356020,NaN,-0.243742,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.295155,NaN,NaN,NaN,NaN
5407344,6_89399126_G_A_b37,ENSG00000111880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
######### DICT with common names from eQTL and RNA-Seq data names #########

In [17]:
tissues_common={}
for b in data.index.unique():
    if b.replace(' - ','_').replace(' ','_').replace('(','').replace(')','') in (eQTLs.columns[2:]):
        #print(b.replace(' - ','_').replace(' ','_').replace('(','').replace(')',''),'matches.')
        tissues_common[b]=b.replace(' - ','_').replace(' ','_').replace('(','').replace(')','')
    else:
        print(b.replace(' - ','_').replace(' ','_').replace('(','').replace(')',''),'doesnt match!!!!!!!!!!!!!!')

Kidney_Cortex doesnt match!!!!!!!!!!!!!!
Fallopian_Tube doesnt match!!!!!!!!!!!!!!
Bladder doesnt match!!!!!!!!!!!!!!
Cervix_Ectocervix doesnt match!!!!!!!!!!!!!!
Cervix_Endocervix doesnt match!!!!!!!!!!!!!!


In [18]:
tissues_common_inv = {v: k for k, v in tissues_common.items()}

In [19]:
tissues_common

{'Adipose - Subcutaneous': 'Adipose_Subcutaneous',
 'Muscle - Skeletal': 'Muscle_Skeletal',
 'Artery - Tibial': 'Artery_Tibial',
 'Artery - Coronary': 'Artery_Coronary',
 'Heart - Atrial Appendage': 'Heart_Atrial_Appendage',
 'Adipose - Visceral (Omentum)': 'Adipose_Visceral_Omentum',
 'Ovary': 'Ovary',
 'Uterus': 'Uterus',
 'Vagina': 'Vagina',
 'Breast - Mammary Tissue': 'Breast_Mammary_Tissue',
 'Skin - Not Sun Exposed (Suprapubic)': 'Skin_Not_Sun_Exposed_Suprapubic',
 'Minor Salivary Gland': 'Minor_Salivary_Gland',
 'Brain - Cortex': 'Brain_Cortex',
 'Adrenal Gland': 'Adrenal_Gland',
 'Thyroid': 'Thyroid',
 'Lung': 'Lung',
 'Spleen': 'Spleen',
 'Pancreas': 'Pancreas',
 'Esophagus - Muscularis': 'Esophagus_Muscularis',
 'Esophagus - Mucosa': 'Esophagus_Mucosa',
 'Esophagus - Gastroesophageal Junction': 'Esophagus_Gastroesophageal_Junction',
 'Stomach': 'Stomach',
 'Colon - Sigmoid': 'Colon_Sigmoid',
 'Small Intestine - Terminal Ileum': 'Small_Intestine_Terminal_Ileum',
 'Colon - Tran

In [ ]:
###### Identify significant (DE) transcripts for each tissue #######

In [116]:
significant={}

In [117]:
for Tissue in data.index.unique(): significant[Tissue]=[]

In [118]:
for Transc in data.columns:
    for Tissue in data.index.unique():
        ttest=ttest_ind(data[Transc][Tissue],data[Transc][data.index != Tissue])
        if ttest.pvalue <= 0.00001:
            significant[Tissue].append(Transc)
            #print(Transc,Tissue,ttest)

In [119]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in significant.items() ]))

,Adipose - Subcutaneous,Muscle - Skeletal,Artery - Tibial,Artery - Coronary,Heart - Atrial Appendage,Adipose - Visceral (Omentum),Ovary,Uterus,Vagina,Breast - Mammary Tissue,...,Brain - Caudate (basal ganglia),Brain - Nucleus accumbens (basal ganglia),Brain - Putamen (basal ganglia),Brain - Hypothalamus,Brain - Spinal cord (cervical c-1),Brain - Amygdala,Fallopian Tube,Bladder,Cervix - Ectocervix,Cervix - Endocervix
0,ENST00000538644.1,ENST00000538644.1,ENST00000538644.1,ENST00000178638.3,ENST00000538644.1,ENST00000256104.4,ENST00000178638.3,ENST00000372216.4,ENST00000178638.3,ENST00000256104.4,...,ENST00000538644.1,ENST00000538644.1,ENST00000178638.3,ENST00000178638.3,ENST00000178638.3,ENST00000178638.3,ENST00000553456.3,ENST00000325888.8,ENST00000006015.3,ENST00000006015.3
1,ENST00000256104.4,ENST00000575363.1,ENST00000256104.4,ENST00000491109.1,ENST00000472523.1,ENST00000472523.1,ENST00000498730.1,ENST00000419268.1,ENST00000498730.1,ENST00000472523.1,...,ENST00000178638.3,ENST00000498730.1,ENST00000498730.1,ENST00000498730.1,ENST00000498730.1,ENST00000498730.1,ENST00000447200.2,ENST00000225538.3,ENST00000559638.1_x,ENST00000296280.6
2,ENST00000472523.1,ENST00000426995.2,ENST00000594907.1,ENST00000297785.3,ENST00000419268.1,ENST00000372216.4,ENST00000491109.1,ENST00000498730.1,ENST00000546916.1,ENST00000498730.1,...,ENST00000498730.1,ENST00000491109.1,ENST00000491109.1,ENST00000491109.1,ENST00000491109.1,ENST00000491109.1,NaN,ENST00000250383.6,ENST00000296280.6,ENST00000553456.3
3,ENST00000284770.5,ENST00000256104.4,ENST00000472523.1,ENST00000448933.1,ENST00000178638.3,ENST00000178638.3,ENST00000297785.3,ENST00000491109.1,ENST00000449873.1,ENST00000491109.1,...,ENST00000491109.1,ENST00000274063.4,ENST00000274063.4,ENST00000297785.3,ENST00000546916.1,ENST00000297785.3,NaN,ENST00000344777.7,ENST00000553456.3,ENST00000249499.6
4,ENST00000178638.3,ENST00000372216.4,ENST00000419268.1,ENST00000375771.1,ENST00000498730.1,ENST00000274063.4,ENST00000343713.4,ENST00000449873.1,ENST00000497741.1,ENST00000274063.4,...,ENST00000274063.4,ENST00000297785.3,ENST00000546916.1,ENST00000546916.1,ENST00000558353.1,ENST00000546916.1,NaN,ENST00000432832.2,ENST00000249499.6,ENST00000510542.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,NaN,ENST00000371100.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,NaN,ENST00000489404.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,NaN,ENST00000523931.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,NaN,ENST00000558184.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
####### Calculate ratio of success for each transcript for each tissue #######

In [120]:
fails={}
success={}
ratio={}

In [131]:
len_tran=len(data.columns)
for Tissue in data.index.unique().sort_values():
    print(Tissue)
    fails[Tissue]=[]
    success[Tissue]=[]
    ratio[Tissue]={}
    for Transc in data.columns:
        print_cum=('Transcript '+str(data.columns.get_loc(Transc)+1)+' of '+str(len_tran)+": ")  
        ttest_tissue=ttest_ind(data[Transc][Tissue],data[Transc][data.index != Tissue])
        if ttest_tissue.pvalue <= 0.00001:
            print_cum=print_cum+'Tissue expression significant:'
            for sample in list((reindex_data[reindex_data['Tissue'] == Tissue]).index):
                print_cum_fin=print_cum+'Sample '+str((reindex_data[reindex_data['Tissue'] == Tissue]).index.get_loc(sample))+' of '+str(len(list((reindex_data[reindex_data['Tissue'] == Tissue]).index)))
                print(print_cum_fin,end="\r",flush=True)
                ttest_sample=stats.ttest_1samp(data[Transc][Tissue],data.iloc[sample][Transc])
                if ttest_sample.pvalue <= 0.0001: success[Tissue].append(Transc)
                elif ttest_sample.pvalue >= 0.0001: fails[Tissue].append(Transc)
                if (data[Transc][Tissue]).mean() == 0 and data.iloc[sample][Transc] == 0:
                    success[Tissue].append(Transc)
                if (data[Transc][Tissue]).mean() == 0 and data.iloc[sample][Transc] != 0:
                    fails[Tissue].append(Transc)
            ratio[Tissue][Transc]=success[Tissue].count(Transc)/(success[Tissue].count(Transc)+fails[Tissue].count(Transc))
        else: ratio[Tissue][Transc]= np.nan
    print(len_tran,"transcripts processed,",len(success[Tissue]),"successful associations",len(fails[Tissue]),"failed. Mean ratio of",pd.Series(ratio[Tissue]).mean(),end="\r",flush=True)
    print("")
    print('###########')

Adipose - Subcutaneous
418 transcripts processed, 26840 successful associations 7194 failed. Mean ratio of 0.7886231415643181
###########
Adipose - Visceral (Omentum)
418 transcripts processed, 20123 successful associations 4017 failed. Mean ratio of 0.8335956917978459
###########
Adrenal Gland
418 transcripts processed, 7867 successful associations 3153 failed. Mean ratio of 0.7138838475499092
###########
Artery - Aorta
418 transcripts processed, 14465 successful associations 4372 failed. Mean ratio of 0.7679035939905505
###########
Artery - Coronary
418 transcripts processed, 4241 successful associations 1468 failed. Mean ratio of 0.7428621474864251
###########
Artery - Tibial
418 transcripts processed, 29598 successful associations 11415 failed. Mean ratio of 0.7216736156828323
###########
Bladder
418 transcripts processed, 6 successful associations 192 failed. Mean ratio of 0.0303030303030303
###########
Brain - Amygdala
418 transcripts processed, 3070 successful associations 1430 

In [159]:
ratiodata=pd.DataFrame(ratio).T

In [160]:
ratiodata.to_csv('Ratiodata_mod_extended.csv',index=True)

In [20]:
ratiodata=pd.read_csv('Ratiodata_mod_extended.csv',index_col=0)

In [209]:
for i in ratiodata.columns:
    if np.isnan(ratiodata[i]).all():
        print(i,'expression is not significant to any tissue.')
    else:
        print(i,'Max:',ratiodata[i].max().round(decimals=2),'Min:',ratiodata[i].min().round(decimals=2))

ENST00000538644.1 Max: 0.91 Min: 0.45
ENST00000575363.1 Max: 0.98 Min: 0.85
ENST00000475226.1 Max: 0.84 Min: 0.84
ENST00000426995.2 Max: 0.87 Min: 0.87
ENST00000398824.2 Max: 0.83 Min: 0.83
ENST00000256104.4 Max: 0.87 Min: 0.23
ENST00000237014.3 Max: 0.84 Min: 0.78
ENST00000594907.1 Max: 0.91 Min: 0.89
ENST00000472523.1 Max: 1.0 Min: 0.06
ENST00000420959.2 Max: 1.0 Min: 0.75
ENST00000372216.4 Max: 0.87 Min: 0.49
ENST00000390315.2 Max: 0.69 Min: 0.02
ENST00000419268.1 Max: 1.0 Min: 0.04
ENST00000468980.2 Max: 0.87 Min: 0.56
ENST00000284770.5 Max: 0.86 Min: 0.63
ENST00000178638.3 Max: 0.89 Min: 0.16
ENST00000522109.1 Max: 0.88 Min: 0.88
ENST00000448599.2 Max: 0.85 Min: 0.79
ENST00000520249.1 Max: 0.85 Min: 0.85
ENST00000498730.1 Max: 0.87 Min: 0.59
ENST00000491109.1 Max: 0.89 Min: 0.61
ENST00000274063.4 Max: 0.87 Min: 0.05
ENST00000297785.3 Max: 0.87 Min: 0.63
ENST00000547869.1 Max: 0.88 Min: 0.63
ENST00000341809.3 Max: 0.86 Min: 0.0
ENST00000343713.4 Max: 0.88 Min: 0.67
ENST00000546916.

In [21]:
for i in ratiodata.columns:
    for b in ratiodata[i].index:
        if ratiodata[i][b]<=0.7:ratiodata[i][b]=np.nan

In [22]:
ratiodata

,ENST00000538644.1,ENST00000575363.1,ENST00000475226.1,ENST00000426995.2,ENST00000398824.2,ENST00000256104.4,ENST00000237014.3,ENST00000594907.1,ENST00000472523.1,ENST00000420959.2,...,ENST00000538107.1,ENST00000377871.3,ENST00000421062.2,ENST00000458333.2,ENST00000528160.1,ENST00000376891.4,ENST00000533330.2,ENST00000341074.5,ENST00000413673.1,ENST00000498496.1
Adipose - Subcutaneous,0.864253,NaN,NaN,NaN,NaN,0.866516,NaN,NaN,0.830317,NaN,...,0.821267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adipose - Visceral (Omentum),NaN,NaN,NaN,NaN,NaN,0.825352,NaN,NaN,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adrenal Gland,0.842105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.994737,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artery - Aorta,0.876254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.816054,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artery - Coronary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artery - Tibial,0.897959,NaN,NaN,NaN,NaN,0.809524,NaN,0.888889,0.834467,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bladder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brain - Amygdala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brain - Anterior cingulate cortex (BA24),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brain - Caudate (basal ganglia),0.793750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
ratiodata.count(axis=1)

Adipose - Subcutaneous                        70
Adipose - Visceral (Omentum)                  64
Adrenal Gland                                 46
Artery - Aorta                                53
Artery - Coronary                             26
Artery - Tibial                               78
Bladder                                        0
Brain - Amygdala                              15
Brain - Anterior cingulate cortex (BA24)      20
Brain - Caudate (basal ganglia)               39
Brain - Cerebellar Hemisphere                 34
Brain - Cerebellum                            47
Brain - Cortex                                29
Brain - Frontal Cortex (BA9)                  34
Brain - Hippocampus                           42
Brain - Hypothalamus                          33
Brain - Nucleus accumbens (basal ganglia)     34
Brain - Putamen (basal ganglia)               30
Brain - Spinal cord (cervical c-1)            15
Brain - Substantia nigra                      14
Breast - Mammary Tis

In [214]:
ratiodata/ratiodata.sum()

,ENST00000538644.1,ENST00000575363.1,ENST00000475226.1,ENST00000426995.2,ENST00000398824.2,ENST00000256104.4,ENST00000237014.3,ENST00000594907.1,ENST00000472523.1,ENST00000420959.2,...,ENST00000538107.1,ENST00000377871.3,ENST00000421062.2,ENST00000458333.2,ENST00000528160.1,ENST00000376891.4,ENST00000533330.2,ENST00000341074.5,ENST00000413673.1,ENST00000498496.1
Adipose - Subcutaneous,0.040108,NaN,NaN,NaN,NaN,0.087717,NaN,NaN,0.052670,NaN,...,0.509675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adipose - Visceral (Omentum),NaN,NaN,NaN,NaN,NaN,0.083550,NaN,NaN,0.063433,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adrenal Gland,0.039080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063100,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artery - Aorta,0.040665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.051765,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artery - Coronary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Artery - Tibial,0.041672,NaN,NaN,NaN,NaN,0.081948,NaN,0.493436,0.052933,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bladder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brain - Amygdala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brain - Anterior cingulate cortex (BA24),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brain - Caudate (basal ganglia),0.036836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
ratiodata.dropna(axis=1,how='all').shape

(53, 367)

In [ ]:
###### Perform predictions on random RNA-seq samples and check performance per tissue #######

In [ ]:
idontknowhowtocalthisval={}

In [ ]:
for Tissue in data.index.unique():
    idontknowhowtocalthisval[Tissue]=[]

In [ ]:
a=eQTLs[tissues_common.values()].rename(columns=tissues_common_inv)
mean_ratio=[]
pred_ratio=[]
slope=[]
weighted_predicted=[]
mean_predicted=[]
tissue=[]
#for sample in random.sample(range(1, len(data.index)), 20000):
for sample in range(1, len(data.index)):
    if data.iloc[sample].name in tissues_common:
        if ~ np.isnan(eQTLs[tissues_common[data.iloc[sample].name]].iloc[sample]):
            print(data.iloc[sample].name)
            test=t_test_ratio_external(data.iloc[sample])
            b=pd.DataFrame(test['value']).sum(axis=1)[tissues_common.keys()]
            ### b_where_a = scores for tissues with eQTL value for such transcript
            b_where_a=b.multiply(~ np.isnan(a))
            b_where_a_sum=b_where_a.sum(axis=1)
            prediction=a.multiply(b_where_a.div(b_where_a_sum,axis=0)).iloc[sample].sum()
            real=eQTLs[tissues_common[data.iloc[sample].name]].iloc[sample]
            mean_prediction=eQTLs[eQTLs.columns[2:].drop(tissues_common[data.iloc[sample].name])].iloc[sample].mean()
            if np.isnan(mean_prediction): continue 
            print('Real/Mean:',real/mean_prediction,'Real/Predicted:',real/prediction)
            mean_ratio.append(real/mean_prediction)
            pred_ratio.append(real/prediction)
            slope.append(real)
            weighted_predicted.append(prediction)
            mean_predicted.append(mean_prediction)
            tissue.append(data.iloc[sample].name)
            print('###############')

In [ ]:
Results=pd.DataFrame({'Tissue':tissue,'Slope':slope,'Predicted':weighted_predicted,'Predicted Ratio':pred_ratio,'Mean':mean_predicted,'Mean Ratio':mean_ratio})

In [ ]:
Results.to_csv('Results0907_extended.csv',index=False)

In [ ]:
####### Random tests from dataset ######

In [178]:
for sample in random.sample(range(1, len(data.index)), 100):
    test=t_test_ratio(sample)
    print(data.iloc[sample].name)
    print(pd.DataFrame(test['value']).sum(axis=1).sort_values(ascending=False)[:5])
    print('###############')

Heart - Atrial Appendage
Heart - Atrial Appendage    4.191919
Liver                       3.514286
Lung                        2.550351
Stomach                     2.423664
Brain - Cortex              2.316456
dtype: float64
###############
Whole Blood
Whole Blood                        9.862408
Brain - Hypothalamus               3.661157
Pancreas                           3.330645
Brain - Caudate (basal ganglia)    2.993750
Heart - Left Ventricle             2.950495
dtype: float64
###############
Adipose - Subcutaneous
Adipose - Subcutaneous     5.882353
Lung                       4.170960
Breast - Mammary Tissue    3.365517
Ovary                      2.909774
Nerve - Tibial             2.541063
dtype: float64
###############
Small Intestine - Terminal Ileum
Stomach                             2.667939
Esophagus - Mucosa                  2.496314
Small Intestine - Terminal Ileum    2.233577
Cells - Transformed fibroblasts     1.755102
Lung                                1.714286
dtyp

Prostate
Colon - Transverse    4.189781
Esophagus - Mucosa    4.159705
Lung                  3.964871
Prostate              3.572368
Colon - Sigmoid       2.287554
dtype: float64
###############
Colon - Sigmoid
Breast - Mammary Tissue    5.658621
Lung                       4.131148
Nerve - Tibial             4.062802
Artery - Tibial            3.541950
Thyroid                    3.434978
dtype: float64
###############
Adrenal Gland
Heart - Atrial Appendage    4.309764
Muscle - Skeletal           3.796099
Brain - Cortex              3.727848
Stomach                     3.400763
Whole Blood                 3.395577
dtype: float64
###############
Esophagus - Muscularis
Artery - Aorta                  3.986622
Artery - Tibial                 3.421769
Thyroid                         3.394619
Adipose - Visceral (Omentum)    3.309859
Testis                          3.235521
dtype: float64
###############
Muscle - Skeletal
Muscle - Skeletal                           9.808511
Stomach           

Adipose - Subcutaneous
Artery - Tibial                 4.219955
Lung                            4.084309
Esophagus - Mucosa              2.594595
Adipose - Visceral (Omentum)    2.538028
Brain - Hippocampus             2.178862
dtype: float64
###############
Liver
Liver                                        7.617143
Brain - Anterior cingulate cortex (BA24)     3.826446
Heart - Left Ventricle                       3.676568
Brain - Putamen (basal ganglia)              3.661290
Brain - Nucleus accumbens (basal ganglia)    3.537415
dtype: float64
###############
Liver
Muscle - Skeletal                  5.828014
Cells - Transformed fibroblasts    5.827988
Liver                              3.777143
Pancreas                           3.201613
Whole Blood                        2.584767
dtype: float64
###############
Brain - Hippocampus
Brain - Nucleus accumbens (basal ganglia)    4.095238
Brain - Cortex                               3.778481
Liver                                        3.76

Spleen
Cells - Transformed fibroblasts    4.072886
Adrenal Gland                      3.847368
Brain - Hypothalamus               3.644628
Brain - Caudate (basal ganglia)    3.612500
Brain - Cortex                     3.575949
dtype: float64
###############
Vagina
Stomach                                4.988550
Colon - Transverse                     4.259124
Skin - Sun Exposed (Lower leg)         3.310782
Skin - Not Sun Exposed (Suprapubic)    3.266150
Cells - Transformed fibroblasts        2.559767
dtype: float64
###############
Artery - Coronary
Artery - Tibial           4.224490
Uterus                    4.009009
Muscle - Skeletal         3.964539
Esophagus - Muscularis    3.310811
Lung                      3.262295
dtype: float64
###############
Brain - Putamen (basal ganglia)
Brain - Cerebellum                           5.687861
Whole Blood                                  3.933661
Heart - Left Ventricle                       3.867987
Brain - Nucleus accumbens (basal ganglia)    3

In [62]:
ratiotest=t_test_ratio(55)

In [63]:
print(data.iloc[55].name)

Adipose - Subcutaneous


In [64]:
pd.DataFrame(ratiotest['value']).sum(axis=1).sort_values(ascending=False)[:5]

Adipose - Subcutaneous             5.929864
Breast - Mammary Tissue            5.034483
Adipose - Visceral (Omentum)       3.957746
Skin - Sun Exposed (Lower leg)     3.350951
Cells - Transformed fibroblasts    3.309038
dtype: float64

In [ ]:
###### Prediction on the mean RNA-seq value per tissue #######

In [24]:
tissue_means=data.groupby('Tissue').mean()

In [25]:
tissue_means

,ENST00000538644.1,ENST00000575363.1,ENST00000475226.1,ENST00000426995.2,ENST00000398824.2,ENST00000256104.4,ENST00000237014.3,ENST00000594907.1,ENST00000472523.1,ENST00000420959.2,...,ENST00000538107.1,ENST00000377871.3,ENST00000421062.2,ENST00000458333.2,ENST00000528160.1,ENST00000376891.4,ENST00000533330.2,ENST00000341074.5,ENST00000413673.1,ENST00000498496.1
Tissue,,,,,,,,,,,,,,,,,,,,,
Adipose - Subcutaneous,0.140475,0.379163,13.724502,0.342308,0.057534,2204.174457,0.664661,0.101516,3.909819,0.057715,...,1.329412,0.000271,0.000136,0.000973,0.017964,0.001787,0.001267,0.043575,0.006629,0.005769
Adipose - Visceral (Omentum),0.719408,0.103437,29.817521,0.251915,0.074873,2461.177746,0.994282,0.076563,0.010648,0.046873,...,1.019746,0.000197,0.000000,0.000535,0.008338,0.002676,0.000930,0.115859,0.011211,0.007775
Adrenal Gland,0.183053,0.087368,4.438158,0.263474,0.040316,9.542368,0.263158,0.054211,0.030316,0.035316,...,0.072789,0.000053,0.000000,0.001263,0.007895,0.001053,0.000789,0.005474,0.006526,0.002842
Artery - Aorta,0.353378,0.114983,23.837291,0.261639,0.097425,28.215953,0.304749,0.076990,0.335151,0.083846,...,1.158428,0.000000,0.000000,0.000836,0.005318,0.004716,0.002575,0.011037,0.009766,0.003612
Artery - Coronary,0.337803,1.470925,20.011792,0.312197,0.098150,247.223699,0.576994,0.086069,0.114740,0.079017,...,1.043931,0.000000,0.000000,0.001272,0.024740,0.002832,0.003988,0.013006,0.006474,0.003699
Artery - Tibial,0.222449,0.119206,20.879909,0.498730,0.050000,37.457370,0.522268,0.236054,0.198231,0.128957,...,0.421837,0.000000,0.000000,0.000748,0.007664,0.005510,0.002222,0.059615,0.005896,0.004036
Bladder,1.184545,0.450000,1.327273,0.353636,0.833636,88.707273,0.460000,0.030909,0.010909,0.183636,...,0.406364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.126364,0.010000,0.000000
Brain - Amygdala,0.220000,0.195300,13.363300,0.118500,0.039400,0.570300,31.129400,0.055900,0.969200,0.030800,...,0.060200,0.024700,0.000400,0.011200,2.599000,0.084500,0.103100,0.006600,0.004100,0.001900
Brain - Anterior cingulate cortex (BA24),0.267769,0.141074,11.073058,0.147851,0.059835,0.522397,10.071488,0.019421,2.674959,0.057769,...,0.049421,0.021901,0.002149,0.013058,1.139504,0.061818,0.053058,0.000579,0.006860,0.008512


In [ ]:
##### Can this be deleted?

In [141]:
for i in tissue_means.index:
    print(i)
    test=t_test_ratio_external(pred_data.loc[i])
    print(pd.DataFrame(test['value']).sum(axis=1).sort_values(ascending=False)[:5])
    print('############################')
    

Adipose - Subcutaneous


NameError: name 'pred_data' is not defined

In [ ]:
####### tests with predictions ####### (Can be deleted?)

In [268]:
pd.DataFrame(test['value']).sum(axis=1).sort_values(ascending=False)[tissues_common.keys()]/pd.DataFrame(test['value']).sum(axis=1).sort_values(ascending=False)[tissues_common.keys()].sum()

Adipose - Subcutaneous                       0.007273
Muscle - Skeletal                            0.030550
Artery - Tibial                              0.013861
Artery - Coronary                            0.000000
Heart - Atrial Appendage                     0.006165
Adipose - Visceral (Omentum)                 0.013768
Ovary                                        0.004606
Uterus                                       0.000000
Vagina                                       0.000000
Breast - Mammary Tissue                      0.000475
Skin - Not Sun Exposed (Suprapubic)          0.006884
Minor Salivary Gland                         0.000000
Brain - Cortex                               0.010761
Adrenal Gland                                0.021992
Thyroid                                      0.011344
Lung                                         0.000742
Spleen                                       0.001020
Pancreas                                     0.019486
Esophagus - Muscularis      

In [26]:
####### Prediction and eQTL comparison for each mean RNA-seq value ######

In [27]:
prediction={}
real={}
mean_prediction={}
### a = renamed eQTLs
a=eQTLs[tissues_common.values()].rename(columns=tissues_common_inv)

for i in tissues_common.keys():
    print(i)
    test=t_test_ratio_external(tissue_means.loc[i])
    ### b = tissue scores according to T test
    b=pd.DataFrame(test['value']).sum(axis=1)[tissues_common.keys()]
    ### b_where_a = scores for tissues with eQTL value for such transcript
    b_where_a=b.multiply(~ np.isnan(a))
    b_where_a_sum=b_where_a.sum(axis=1)
    prediction[i]=a.multiply(b_where_a.div(b_where_a_sum,axis=0)).sum(axis=1)
    #prediction[i]=(eQTLs[tissues_common.values()].rename(columns=tissues_common_inv)*pd.DataFrame(test['value']).sum(axis=1)[tissues_common.keys()]/pd.DataFrame(test['value']).sum(axis=1)[tissues_common.keys()].sum()).sum(axis=1)
    real[i]=eQTLs[tissues_common[i]]
    mean_prediction[i]=eQTLs[eQTLs.columns[2:].drop(tissues_common[i])].mean(axis=1)
    #print(pd.DataFrame(test['value']).sum(axis=1).sort_values(ascending=False)[:5])
    print('############################')

Adipose - Subcutaneous
############################
Muscle - Skeletal
############################
Artery - Tibial
############################
Artery - Coronary
############################
Heart - Atrial Appendage
############################
Adipose - Visceral (Omentum)
############################
Ovary
############################
Uterus
############################
Vagina
############################
Breast - Mammary Tissue
############################
Skin - Not Sun Exposed (Suprapubic)
############################
Minor Salivary Gland
############################
Brain - Cortex
############################
Adrenal Gland
############################
Thyroid
############################
Lung
############################
Spleen
############################
Pancreas
############################
Esophagus - Muscularis
############################
Esophagus - Mucosa
############################
Esophagus - Gastroesophageal Junction
############################
Stomach
###############

In [28]:
Predicted=pd.DataFrame(prediction)

In [29]:
Real=pd.DataFrame(real)

In [30]:
Mean_predicted=pd.DataFrame(mean_prediction)

In [31]:
Predicted

,Adipose - Subcutaneous,Muscle - Skeletal,Artery - Tibial,Artery - Coronary,Heart - Atrial Appendage,Adipose - Visceral (Omentum),Ovary,Uterus,Vagina,Breast - Mammary Tissue,...,Brain - Substantia nigra,Brain - Anterior cingulate cortex (BA24),Brain - Frontal Cortex (BA9),Brain - Cerebellar Hemisphere,Brain - Caudate (basal ganglia),Brain - Nucleus accumbens (basal ganglia),Brain - Putamen (basal ganglia),Brain - Hypothalamus,Brain - Spinal cord (cervical c-1),Brain - Amygdala
0,0.272490,0.352267,0.241380,0.270059,0.299464,0.260827,0.273474,0.265026,0.274258,0.266117,...,0.305135,0.320509,0.303870,0.297907,0.305495,0.309562,0.307673,0.297591,0.299098,0.309119
1,-0.120414,-0.156478,-0.147481,-0.137511,-0.178002,-0.137301,-0.153529,-0.166476,-0.164407,-0.149603,...,-0.182470,-0.186509,-0.195418,-0.174261,-0.174648,-0.171896,-0.172789,-0.181603,-0.151636,-0.200296
2,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,...,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124
3,0.724702,0.727761,0.726379,0.723728,0.726482,0.723607,0.725161,0.724616,0.724276,0.723247,...,0.727059,0.724209,0.722316,0.723991,0.727825,0.723251,0.724776,0.725720,0.722797,0.725824
4,0.161757,0.151456,0.157442,0.175218,0.157188,0.158756,0.159280,0.169650,0.172962,0.161145,...,0.144726,0.149882,0.170923,0.142377,0.144298,0.142245,0.140841,0.144022,0.147086,0.143613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407341,-0.151082,-0.151801,-0.151610,-0.151417,-0.155249,-0.152907,-0.156318,-0.153577,-0.153164,-0.151803,...,-0.156622,-0.151069,-0.153285,-0.150088,-0.152148,-0.151032,-0.151958,-0.151770,-0.152647,-0.149221
5407342,-0.253003,-0.250063,-0.276675,-0.270852,-0.286506,-0.262781,-0.282734,-0.287734,-0.278229,-0.270989,...,-0.290790,-0.292604,-0.287392,-0.284201,-0.293094,-0.287929,-0.300225,-0.280846,-0.286123,-0.291506
5407343,-0.288884,-0.274047,-0.263043,-0.290115,-0.278165,-0.295010,-0.288019,-0.293355,-0.296085,-0.291646,...,-0.273878,-0.275870,-0.283889,-0.283844,-0.281094,-0.279096,-0.275845,-0.280012,-0.285374,-0.272524
5407344,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,...,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984


In [32]:
Mean_predicted

,Adipose - Subcutaneous,Muscle - Skeletal,Artery - Tibial,Artery - Coronary,Heart - Atrial Appendage,Adipose - Visceral (Omentum),Ovary,Uterus,Vagina,Breast - Mammary Tissue,...,Brain - Substantia nigra,Brain - Anterior cingulate cortex (BA24),Brain - Frontal Cortex (BA9),Brain - Cerebellar Hemisphere,Brain - Caudate (basal ganglia),Brain - Nucleus accumbens (basal ganglia),Brain - Putamen (basal ganglia),Brain - Hypothalamus,Brain - Spinal cord (cervical c-1),Brain - Amygdala
0,0.285706,0.274217,0.291724,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726,...,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726,0.283726
1,-0.213037,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,...,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307,-0.177307
2,0.136124,NaN,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,...,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124,0.136124
3,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,...,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376,0.727376
4,0.169599,0.169599,0.174484,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599,...,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599,0.169599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407341,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,...,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019,-0.154019
5407342,-0.305262,-0.306744,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,...,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249,-0.298249
5407343,-0.290932,-0.290932,-0.298797,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,...,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932,-0.290932
5407344,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,...,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984,-0.269984


In [33]:
Real_mean_corr={}
Real_pred_corr={}
for i in Real.columns:
    Real_mean_corr[i]=Real[i].corr(Mean_predicted[i])
    Real_pred_corr[i]=Real[i].corr(Predicted[i])

In [34]:
#pd.DataFrame.from_dict(Real_mean_corr,orient='index')

In [35]:
Results_means=pd.DataFrame([Real_mean_corr,Real_pred_corr],index=['Real-Mean corr','Real-Pred corr']).T

In [36]:
Results_means['Difference']=Results_means['Real-Pred corr']-Results_means['Real-Mean corr']

In [37]:
Results_means['Difference'].mean()

0.04440248029902921

In [38]:
Results_means

,Real-Mean corr,Real-Pred corr,Difference
Adipose - Subcutaneous,0.961324,0.994002,0.032679
Muscle - Skeletal,0.891832,0.991609,0.099776
Artery - Tibial,0.949930,0.993619,0.043689
Artery - Coronary,0.978079,0.990721,0.012642
Heart - Atrial Appendage,0.955623,0.990477,0.034854
Adipose - Visceral (Omentum),0.968499,0.993106,0.024607
Ovary,0.968507,0.984236,0.015729
Uterus,0.974668,0.979636,0.004968
Vagina,0.973491,0.984726,0.011234
Breast - Mammary Tissue,0.974652,0.992110,0.017457


In [39]:
Results_means.merge(pd.DataFrame(ratiodata.count(axis=1),columns=['Sig Transc']),how='left',left_index=True,right_index=True).merge(pd.DataFrame((ratiodata/ratiodata.sum()).mean(axis=1),columns=['Mean sig transc score']),how='left',left_index=True,right_index=True).to_csv('Results_to_plot_1007.csv',index=True)

In [40]:
pd.DataFrame(ratiodata.count(axis=1),columns=['Sig Transc'])

,Sig Transc
Adipose - Subcutaneous,70
Adipose - Visceral (Omentum),64
Adrenal Gland,46
Artery - Aorta,53
Artery - Coronary,26
Artery - Tibial,78
Bladder,0
Brain - Amygdala,15
Brain - Anterior cingulate cortex (BA24),20
Brain - Caudate (basal ganglia),39


In [41]:
Results_means['Difference'].mean()

0.04440248029902921